In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/dataset'

In [ ]:
import pandas as pd
import spacy
from spacy.tokens import DocBin
import numpy as np

In [ ]:
spacy.__version__

'2.3.2'

In [ ]:
!pip install spacy==2.3.2

     |████████████████████████████████| 9.9 MB 13.0 MB/s 
     |████████████████████████████████| 2.1 MB 62.1 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
doc_bin = DocBin(attrs=["LEMMA", "ENT_IOB", "ENT_TYPE"], store_user_data=True)
nlp2 = spacy.load("/content/drive/MyDrive/Colab Notebooks/dataset/models/v2")


# **Data Preprocessing**

In [ ]:
df =pd.read_csv('clean.csv')

In [ ]:
df.head()

,Unnamed: 0,address,name,rate,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,type,city,Mean Rating
0,7144,"2334, Aarush Arcade, 17th Cross, 1st Sector, H...",Le Charcoal Xpress,4.0,164,HSR,Quick Bites,"Pita Bread, Rolls, Shawarma Roll, Lebanese Sha...","Arabian, BBQ, Rolls, Lebanese",500.0,"[(' .', ' Great food. Had alfaham, one shawa...",Delivery,HSR,3.67
1,20162,"#263, Domlur 2nd Stage, Indiranagar, Bangalore",Lavonne,4.4,1130,Indiranagar,"Cafe, Dessert Parlor","Coffee, Burgers, Croissants, Cheesecake, Pasta...","Cafe, Desserts",800.0,"[(' .', ' One of those places which desrves ...",Cafes,Old Airport Road,4.35
2,21878,"5th Floor, Confident Hotel, Hosur Main Road, L...",ThrowBack - Pub & Dine,3.9,235,Langford Town,"Bar, Casual Dining","Cocktails, Pizza, French Fries","Italian, North Indian, Continental",1.0,"[(' .', "" Accidently had visited the bar, co...",Dine-out,Residency Road,3.71
3,22838,"54, Ground Floor, The Planet, Opposite Hindust...",Rolls On Wheels,3.1,76,Whitefield,Quick Bites,"Rolls, Momos, Fries, Chicken Manchurian Roll","Fast Food, Rolls, Momos",300.0,"[(' .', ' This place offers delicious rolls!...",Delivery,Whitefield,2.89
4,3406,"601, 60 Feet Main Raod, AECS Layout, Kundalaha...",Amaravati Authentic Andhra Restaurant,3.0,148,Brookefield,Casual Dining,"Thali, Chilli Chicken, Chicken Curry, Andhra M...","Andhra, Chinese, North Indian, Biryani",600.0,"[(' .', ' Food is worst dont go for it. Tri...",Dine-out,Brookefield,2.57


In [ ]:
review = (df[['name','reviews_list']].set_index(['name'])
   .apply(lambda x: x.str.split(r'\)\,').explode())
   .reset_index()) 

In [ ]:
review['reviews_list']=review['reviews_list'].str.replace('[', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
review['reviews_list']=review['reviews_list'].str.replace(r'\(\' \.\'\, \'','',regex=True)

In [ ]:
review['reviews_list']=review['reviews_list'].str.replace(r'\(\' \.\'\, \"','',regex=True)

In [ ]:
review['reviews_list']=review['reviews_list'].str.replace(r'\"\)\]','',regex=True)

In [ ]:
review['reviews_list']=review['reviews_list'].str.replace('\'', '')

In [ ]:
review['reviews_list']=review['reviews_list'].str.replace('\*', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
review['reviews_list'].drop_duplicates(inplace=True)

In [ ]:
review.reset_index(inplace=True)

In [ ]:
review['name&num'] =review['name']+', '+review['index'].astype(str)

In [ ]:
review2 =review[["name&num","reviews_list"]]

In [ ]:
review_new = (review2.set_index(['name&num'])
   .apply(lambda x: x.str.split('.').explode())
   .reset_index())

In [ ]:
review_new

,name&num,reviews_list
0,"Le Charcoal Xpress, 0",Great food
1,"Le Charcoal Xpress, 0","Had alfaham, one shawarma plate and falafel"
2,"Le Charcoal Xpress, 0",All crash and tastes really good
3,"Le Charcoal Xpress, 0",Prime place in hSR right next to MK retail ma...
4,"Le Charcoal Xpress, 1",Bad
...,...,...
4555675,"Kaafi Desi, 651177",
4555676,"Kaafi Desi, 651177",go for it
4555677,"Kaafi Desi, 651177",
4555678,"Kaafi Desi, 651177",


In [ ]:
review_new['reviews_list'].replace('', np.nan, inplace=True)

In [ ]:
review_new.dropna(inplace=True)

In [ ]:
review_new.to_csv('split_review.csv')

# **Find Food Entity from review**

In [ ]:
#sampling times 100
newfood =review.sample(n=38400)

In [ ]:
newfood

,index,name,reviews_list,name&num
386145,386145,East Delicious,My all time favorite chinese station Ã\xÃ...,"East Delicious, 386145"
219663,219663,Koshy's,We entered KoshyÃ\xÃ\xÃ\xÃ\xÃ\xÃ\xÃ...,"Koshy's, 219663"
530479,530479,Hammered,I visited on Friday (yesterday) with a fri...,"Hammered, 530479"
213190,213190,Hatti Punjab Di,"Centrally located, small restaurant with f...","Hatti Punjab Di, 213190"
72097,72097,Zamindar,Well i wanted to give starS but the soup ...,"Zamindar, 72097"
...,...,...,...,...
573743,573743,Chianti,Visited this restaurant today evening and ...,"Chianti, 573743"
34167,34167,Roots,This is may FAV place in Koramangala when ...,"Roots, 34167"
525162,525162,Chinita Real Mexican Food,Fresh ingredients; courteous and smiling s...,"Chinita Real Mexican Food, 525162"
260695,260695,Kusum Restaurant,This place is very unhygienic! They cook m...,"Kusum Restaurant, 260695"


In [ ]:
newfood['name&num'] =newfood['name']+', '+newfood['index'].astype(str)

In [ ]:
newfood =newfood[["name&num","reviews_list"]]

In [ ]:
newfood= (newfood.set_index(['name&num'])
   .apply(lambda x: x.str.split('.').explode())
   .reset_index())

In [ ]:
newfood

,name&num,reviews_list
0,"East Delicious, 386145",My all time favorite chinese station Ã\xÃ...
1,"East Delicious, 386145",Crowd is very nice and place is very much clean
2,"East Delicious, 386145",Noodles are delicious here
3,"East Delicious, 386145",
4,"Koshy's, 219663",We entered KoshyÃ\xÃ\xÃ\xÃ\xÃ\xÃ\xÃ...
...,...,...
268724,"Kusum Restaurant, 260695",
268725,"Kusum Restaurant, 260695",Its not clean enough
268726,"Kusum Restaurant, 260695",
268727,"Kusum Restaurant, 260695",This has to be your last option to dine :)


In [ ]:
def all_ents(v):
        return [(ent.text, ent.label_) for ent in nlp2(v).ents]

newfood['Entities'] = newfood['reviews_list'].apply(lambda v: all_ents(v))

def food(v):
    r = [(x,y) for x,y in v if ('FOOD' == y)]
    return [(x) for x,y in r]

newfood['FOOD'] = newfood['Entities'].apply(lambda v: food(v))

In [ ]:
newfood

,name&num,reviews_list,Entities,FOOD
0,"East Delicious, 386145",My all time favorite chinese station Ã\xÃ...,"[(chinese station, FOOD), (min, TIME)]",[chinese station]
1,"East Delicious, 386145",Crowd is very nice and place is very much clean,[],[]
2,"East Delicious, 386145",Noodles are delicious here,[],[]
3,"East Delicious, 386145",,[],[]
4,"Koshy's, 219663",We entered KoshyÃ\xÃ\xÃ\xÃ\xÃ\xÃ\xÃ...,[(KoshyÃ\xÃ\xÃ\xÃ\xÃ\xÃ\xÃ\xÃÂ¢Ã\xÃ\...,[]
...,...,...,...,...
268724,"Kusum Restaurant, 260695",,[],[]
268725,"Kusum Restaurant, 260695",Its not clean enough,[],[]
268726,"Kusum Restaurant, 260695",,[],[]
268727,"Kusum Restaurant, 260695",This has to be your last option to dine :),[],[]


# **Sentiment Analysis**

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text.freqdist import FreqDistVisualizer
from yellowbrick.style import set_palette

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
com=[]

In [ ]:
# getting only the negative score
def negative_score(text):
    negative_value = analyzer.polarity_scores(text)['neg']
    neg.append(negative_value)

# getting only the neutral score
def neutral_score(text):
    neutral_value = analyzer.polarity_scores(text)['neu']
    neu.append(neutral_value)

# getting only the positive score
def positive_score(text):
    positive_value = analyzer.polarity_scores(text)['pos']
    pos.append(positive_value)

# getting only the compound score
def compound_score(text):
    compound_value = analyzer.polarity_scores(text)['compound']
    com.append(compound_value) 

In [ ]:
#calculate the score
for i in range (len(newfood)):
  compound_score(newfood['reviews_list'].iloc[i])

In [ ]:
#save the scores into dataframe
newfood['sentiment_compound'] = com

In [ ]:
newfood

,name&num,reviews_list,Entities,FOOD,sentiment_compound
0,"East Delicious, 386145",My all time favorite chinese station Ã\xÃ...,"[(chinese station, FOOD), (min, TIME)]",[chinese station],0.7717
1,"East Delicious, 386145",Crowd is very nice and place is very much clean,[],[],0.7245
2,"East Delicious, 386145",Noodles are delicious here,[],[],0.5719
3,"East Delicious, 386145",,[],[],0.0000
4,"Koshy's, 219663",We entered KoshyÃ\xÃ\xÃ\xÃ\xÃ\xÃ\xÃ...,[(KoshyÃ\xÃ\xÃ\xÃ\xÃ\xÃ\xÃ\xÃÂ¢Ã\xÃ\...,[],0.0000
...,...,...,...,...,...
268724,"Kusum Restaurant, 260695",,[],[],0.0000
268725,"Kusum Restaurant, 260695",Its not clean enough,[],[],-0.3089
268726,"Kusum Restaurant, 260695",,[],[],0.0000
268727,"Kusum Restaurant, 260695",This has to be your last option to dine :),[],[],0.4588


In [ ]:
newfood.to_csv("100_sampling(NER,SC).csv")

# **Part of Speech from review**

In [ ]:
new1=pd.read_csv("100_sampling(NER,SC).csv")

In [ ]:
new1.isna().sum()

Unnamed: 0                0
name&num                  0
reviews_list          59191
Entities                  0
FOOD                      0
sentiment_compound        0
dtype: int64

In [ ]:
new1.dropna(inplace = True)

In [ ]:
new1.reset_index(inplace=True)

In [ ]:
new1['reviews_list'] = new1['reviews_list'].str.replace(r'[^\w\s]+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))


# sent_tokenize is one of instances of 
# PunktSentenceTokenizer from the nltk.tokenize.punkt module
taglist=[]
tag=[]
for i in range(len(new1)):
      
    # Word tokenizers is used to find the words 
    # and punctuation in a string
    wordsList = nltk.word_tokenize(new1['reviews_list'].loc[i])
  
    # removing stop words from wordList
    wordsList = [w for w in wordsList if not w in stop_words] 
  
    #  Using a Tagger. Which is part-of-speech 
    # tagger or POS-tagger. 
    tagged = nltk.pos_tag(wordsList)
    tag.append(tagged)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
new1['Tag']=tag

In [ ]:
new1['POS'] =tag

In [ ]:
for i in range(len(new1)):
  temp=[]
  temp=new1['POS'].loc[i]
  temp2=[]
  for j in range(len(new1['POS'].loc[i])): 
    temp2.append(temp[j][1])
    if j == len(new1['POS'].loc[i])-1:
      new1['Tag'].loc[i] = temp2

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
new1.to_csv('TAG_sampling_100.csv')

In [ ]:
new1 =pd.read_csv('TAG_sampling_100.csv')

In [ ]:
new1.isna().sum()

Unnamed: 0               0
index                    0
Unnamed: 0.1             0
name&num                 0
reviews_list          7490
Entities                 0
FOOD                     0
sentiment_compound       0
Tag                      0
POS                      0
dtype: int64

In [ ]:
new1.dropna(inplace=True)

In [ ]:
new1['FOOD']=new1['FOOD'].str.replace(r'\[','',regex=True)

In [ ]:
new1['FOOD']=new1['FOOD'].str.replace(r'\]','',regex=True)

In [ ]:
new1['Tag']=new1['Tag'].str.replace(r'\[','',regex=True)
new1['Tag']=new1['Tag'].str.replace(r'\]','',regex=True)

In [ ]:
new1['Got FOOD'] = new1['FOOD']

In [ ]:
new1.to_csv('TAG_sampling_100_ver2.csv')

In [ ]:
new1 =pd.read_csv('TAG_sampling_100_ver2.csv')

In [ ]:
new1['Got FOOD'].loc[~new1['Got FOOD'].isnull()] = 1  # not nan
new1['Got FOOD'].loc[new1['Got FOOD'].isnull()] = 0   # nan

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
new1['Sentiment'] = new1['sentiment_compound']

In [ ]:
new1['sentiment_compound'].loc[new1['sentiment_compound'] >0] = 1  #positive
new1['sentiment_compound'].loc[new1['sentiment_compound'] ==0] = 0  #neutral
new1['sentiment_compound'].loc[new1['sentiment_compound'] <0] = -1 #negative

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
new1.tail(100)

,Unnamed: 0,Unnamed: 0.1,index,Unnamed: 0.1.1,name&num,reviews_list,Entities,FOOD,sentiment_compound,Tag,POS,Got FOOD,Sentiment
201948,209434,209434,268500,268500,"Communiti, 67611",We have ordered Almost most of the drinks from...,[],NaN,-1.0,"'PRP', 'VBD', 'NNP', 'NNS', 'VBP', 'VBN', 'NNS...","[('We', 'PRP'), ('ordered', 'VBD'), ('Almost',...",0,-0.3818
201949,209435,209435,268501,268501,"Communiti, 67611",Recommendedfish with lemon caper sauce Peper ...,"[('lemon caper sauce', 'FOOD'), ('Peper', 'FOO...","'lemon caper sauce', 'Peper', 'BBQ wings', 'bu...",0.0,"'JJ', 'NN', 'NN', 'NN', 'NNP', 'NN', 'NNP', 'N...","[('Recommendedfish', 'JJ'), ('lemon', 'NN'), (...",1,0.0000
201950,209436,209436,268503,268503,"Koramangala Social, 138504",And then like most of the SOCIAL outlets n...,"[('Koramangala', 'ORG')]",NaN,1.0,"'CC', 'IN', 'NNP', 'NNS', 'JJ', 'NNP', 'NNP', ...","[('And', 'CC'), ('like', 'IN'), ('SOCIAL', 'NN...",0,0.3612
201951,209437,209437,268504,268504,"Koramangala Social, 138504",The ambience is well known hence nothing much...,[],NaN,1.0,"'DT', 'NN', 'RB', 'VBN', 'NN', 'NN', 'JJ', 'NN'","[('The', 'DT'), ('ambience', 'NN'), ('well', '...",0,0.2732
201952,209438,209438,268505,268505,"Koramangala Social, 138504",You can try every now and then with friends a...,[],NaN,1.0,"'PRP', 'VBP', 'DT', 'NNS', 'JJ', 'NNS'","[('You', 'PRP'), ('try', 'VBP'), ('every', 'DT...",0,0.7003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202043,209533,209533,268723,268723,"Kusum Restaurant, 260695",Bettet to avoid Jan The rolls are not ba...,"[('Bettet', 'PERSON')]",NaN,1.0,"'NNP', 'VBZ', 'NNP', 'DT', 'NNS', 'JJ', 'VBP',...","[('Bettet', 'NNP'), ('avoid', 'VBZ'), ('Jan', ...",0,0.6340
202044,209534,209534,268724,268724,"Kusum Restaurant, 260695",,[],NaN,0.0,NaN,[],0,0.0000
202045,209535,209535,268725,268725,"Kusum Restaurant, 260695",Its not clean enough,[],NaN,-1.0,"'PRP$', 'JJ', 'RB'","[('Its', 'PRP$'), ('clean', 'JJ'), ('enough', ...",0,-0.3089
202046,209536,209536,268727,268727,"Kusum Restaurant, 260695",This has to be your last option to dine,[],NaN,1.0,"'DT', 'JJ', 'NN', 'NN'","[('This', 'DT'), ('last', 'JJ'), ('option', 'N...",0,0.4588


In [ ]:
try1=new1.loc[new1['Got FOOD']== 1] #get only review with food entity 

In [ ]:
try1=try1.groupby(['Tag','sentiment_compound']).count()

In [ ]:
try1=try1.sort_values(['index'],ascending=False)

In [ ]:
try1.head(15)

,,Unnamed: 0,Unnamed: 0.1,index,Unnamed: 0.1.1,name&num,reviews_list,Entities,FOOD,POS,Got FOOD,Sentiment
Tag,sentiment_compound,,,,,,,,,,,
"'NNP', 'NN', 'NN'",0.0,94,94,94,94,94,94,94,94,94,94,94
"'NNP', 'NN', 'NN', 'NN'",0.0,81,81,81,81,81,81,81,81,81,81,81
"'DT', 'NN', 'NN', 'NN'",0.0,64,64,64,64,64,64,64,64,64,64,64
"'NNP', 'NN', 'NN', 'NN', 'NN'",0.0,55,55,55,55,55,55,55,55,55,55,55
"'NN', 'NN', 'NN'",0.0,52,52,52,52,52,52,52,52,52,52,52
"'NNP', 'JJ', 'NN', 'NN'",0.0,50,50,50,50,50,50,50,50,50,50,50
"'DT', 'NN', 'NN', 'JJ'",1.0,43,43,43,43,43,43,43,43,43,43,43
"'NNP', 'NN', 'NN'",1.0,42,42,42,42,42,42,42,42,42,42,42
"'NNP', 'NN', 'NN', 'NN'",1.0,42,42,42,42,42,42,42,42,42,42,42


In [ ]:
try1.to_csv('Sentiment_FOOD_100.csv')

# **Split Result into Positive, Neutral, and Negative category**

In [ ]:
sampling2 = pd.read_csv('Sentiment_FOOD_100.csv')

In [ ]:
positive2 =sampling2.loc[sampling2['sentiment_compound']== 1]
positive2

,Tag,sentiment_compound,Unnamed: 0,Unnamed: 0.1,index,Unnamed: 0.1.1,name&num,reviews_list,Entities,FOOD,POS,Got FOOD,Sentiment
6,"'DT', 'NN', 'NN', 'JJ'",1.0,43,43,43,43,43,43,43,43,43,43,43
7,"'NNP', 'NN', 'NN'",1.0,42,42,42,42,42,42,42,42,42,42,42
8,"'NNP', 'NN', 'NN', 'NN'",1.0,42,42,42,42,42,42,42,42,42,42,42
9,"'DT', 'NN', 'NN', 'NN'",1.0,41,41,41,41,41,41,41,41,41,41,41
15,"'DT', 'NN', 'NN', 'RB', 'JJ'",1.0,35,35,35,35,35,35,35,35,35,35,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24572,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'NN', 'JJ', 'NN...",1.0,1,1,1,1,1,1,1,1,1,1,1
24573,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'NN', 'JJ', 'NN'",1.0,1,1,1,1,1,1,1,1,1,1,1
24574,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'JJ', 'NN', 'NN...",1.0,1,1,1,1,1,1,1,1,1,1,1
24575,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'JJ', 'NN', 'NN...",1.0,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
negative2 =sampling2.loc[sampling2['sentiment_compound']== -1]
negative2

,Tag,sentiment_compound,Unnamed: 0,Unnamed: 0.1,index,Unnamed: 0.1.1,name&num,reviews_list,Entities,FOOD,POS,Got FOOD,Sentiment
98,"'DT', 'NN', 'NN', 'NN', 'NN'",-1.0,11,11,11,11,11,11,11,11,11,11,11
138,"'NN', 'PRP', 'VBD', 'JJ', 'NN', 'NN', 'NNS', '...",-1.0,10,10,10,10,10,10,10,10,10,10,10
145,"'NNP', 'JJ', 'NN', 'VBN', 'NN', 'NN', 'NN', 'J...",-1.0,9,9,9,9,9,9,9,9,9,9,9
153,"'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP...",-1.0,9,9,9,9,9,9,9,9,9,9,9
167,"'PRP', 'VBP', 'JJ', 'NNS', 'VBP'",-1.0,9,9,9,9,9,9,9,9,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24543,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'NN', 'RB', 'VBD'",-1.0,1,1,1,1,1,1,1,1,1,1,1
24547,"'NNP', 'JJ', 'NN', 'NN', 'JJ', 'NNS', 'IN', 'J...",-1.0,1,1,1,1,1,1,1,1,1,1,1
24550,"'NNP', 'JJ', 'NN', 'NN', 'JJ', 'NN', 'VB'",-1.0,1,1,1,1,1,1,1,1,1,1,1
24561,"'NNP', 'JJ', 'NN', 'NN', 'NN'",-1.0,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
neural2 =sampling2.loc[sampling2['sentiment_compound']== 0]
neural2

,Tag,sentiment_compound,Unnamed: 0,Unnamed: 0.1,index,Unnamed: 0.1.1,name&num,reviews_list,Entities,FOOD,POS,Got FOOD,Sentiment
0,"'NNP', 'NN', 'NN'",0.0,94,94,94,94,94,94,94,94,94,94,94
1,"'NNP', 'NN', 'NN', 'NN'",0.0,81,81,81,81,81,81,81,81,81,81,81
2,"'DT', 'NN', 'NN', 'NN'",0.0,64,64,64,64,64,64,64,64,64,64,64
3,"'NNP', 'NN', 'NN', 'NN', 'NN'",0.0,55,55,55,55,55,55,55,55,55,55,55
4,"'NN', 'NN', 'NN'",0.0,52,52,52,52,52,52,52,52,52,52,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24563,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'NN', 'RB'",0.0,1,1,1,1,1,1,1,1,1,1,1
24564,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN...",0.0,1,1,1,1,1,1,1,1,1,1,1
24568,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'NN', 'NN', 'JJ...",0.0,1,1,1,1,1,1,1,1,1,1,1
24570,"'NNP', 'JJ', 'NN', 'NN', 'NN', 'IN', 'NN'",0.0,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
neural2.to_csv('neural_100.csv')
positive2.to_csv('positive_100.csv')
negative2.to_csv('negative_100.csv')